In [29]:
# Import required libraries
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from flask import Flask, request, jsonify
import joblib

In [30]:
# Function to download historical stock data from Yahoo Finance
def download_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

# Function to preprocess stock data
def preprocess_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data[['Open', 'High', 'Low', 'Close', 'Volume']])
    return scaled_data, scaler

# Function to create sequences for LSTM model
def create_sequences(data, sequence_length):
    sequences = []
    target = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i + sequence_length]
        label = data[i + sequence_length][3]  # Closing price as the target
        sequences.append(seq)
        target.append(label)
    return np.array(sequences), np.array(target)

# Function to build and train LSTM model
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Function to train the LSTM model
def train_model(X_train, y_train, epochs=50, batch_size=32):
    model = build_lstm_model((X_train.shape[1], X_train.shape[2]))
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
    return model

# Function to evaluate the model on test data
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    return mse

In [31]:
# Flask application for prediction
app = Flask(__name__)

# Endpoint for predicting stock prices
@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    input_data = np.array(data['input_data'])
    input_data = input_data.reshape(1, input_data.shape[0], input_data.shape[1])
    prediction = model.predict(input_data)
    return jsonify({'prediction': prediction.tolist()})

In [33]:
if __name__ == "__main__":
    # Download historical stock data
    ticker = "AAPL"
    start_date = "2022-01-01"
    end_date = "2023-01-01"
    stock_data = download_stock_data(ticker, start_date, end_date)

    # Preprocess data
    scaled_data, scaler = preprocess_data(stock_data)

    # Create sequences for LSTM model
    sequence_length = 10
    X, y = create_sequences(scaled_data, sequence_length)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the LSTM model
    model = train_model(X_train, y_train)

    # Evaluate the model
    mse = evaluate_model(model, X_test, y_test)
    print(f"Mean Squared Error on Test Data: {mse}")

    # Save the model and scaler
    joblib.dump(scaler, 'scaler.pkl')
    model.save('stock_prediction_model.h5')

[*********************100%%**********************]  1 of 1 completed
Epoch 1/50
6/6 [==============================] - 7s 14ms/step - loss: 0.1138
Epoch 2/50
6/6 [==============================] - 0s 14ms/step - loss: 0.0316
Epoch 3/50
6/6 [==============================] - 0s 19ms/step - loss: 0.0174
Epoch 4/50
6/6 [==============================] - 0s 18ms/step - loss: 0.0208
Epoch 5/50
6/6 [==============================] - 0s 17ms/step - loss: 0.0159
Epoch 6/50
6/6 [==============================] - 0s 17ms/step - loss: 0.0148
Epoch 7/50
6/6 [==============================] - 0s 16ms/step - loss: 0.0136
Epoch 8/50
6/6 [==============================] - 0s 20ms/step - loss: 0.0132
Epoch 9/50
6/6 [==============================] - 0s 18ms/step - loss: 0.0124
Epoch 10/50
6/6 [==============================] - 0s 15ms/step - loss: 0.0123
Epoch 11/50
6/6 [==============================] - 0s 16ms/step - loss: 0.0122
Epoch 12/50
6/6 [==============================] - 0s 17ms/step - loss:

c:\Users\admin\Desktop\Assignment2\venv\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [34]:
# Run Flask application
app.run(debug=True, port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\admin\Desktop\Assignment2\venv\lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
